## Here I try to reproduce the results from the professor, even if he has made some mistakes.

In [1]:
import math
import numpy as np
import numpy.linalg as la
import pandas as pd
import matplotlib.pyplot as plt

data_file = pd.ExcelFile("NYU_Spring_2019_Assignment_3_data.xls")
#read first sheet
bond_index = pd.read_excel(data_file, "VBTLX", skiprows=[0])
columns = bond_index.columns.values
columns[0] = "Year"
#read second sheet
int_growth = pd.read_excel(data_file, "VWILX", skiprows=[0])
columns = int_growth.columns.values
columns[0] = "Year"
#read third sheet
metal = pd.read_excel(data_file, "VGPMX", skiprows=[0])
columns = metal.columns.values
columns[0] = "Year"
#read fourth sheet
stock_index = pd.read_excel(data_file, "VTSAX", skiprows=[0])
columns = stock_index.columns.values
columns[0] = "Year"
#read fifth sheet
int_value = pd.read_excel(data_file, "VTRIX", skiprows=[0])
columns = int_value.columns.values
columns[0] = "Year"
#read sixth sheet
reit = pd.read_excel(data_file, "VGSLX", skiprows=[0])
columns = reit.columns.values
columns[0] = "Year"
#read sixth sheet
money_market = pd.read_excel(data_file, "VMRXX", skiprows=[0])
columns = money_market.columns.values
columns[0] = "Year"

# Question 1, Calculate excess return
This is done by element wise substraction of risk-free rate.

In [2]:
bond_index_excess = bond_index[["Q1", "Q2", "Q3", "Q4"]] - money_market[["Q1", "Q2", "Q3","Q4"]]
int_growth_excess = int_growth[["Q1", "Q2", "Q3", "Q4"]] - money_market[["Q1", "Q2", "Q3","Q4"]]
metal_excess = metal[["Q1", "Q2", "Q3", "Q4"]] - money_market[["Q1", "Q2", "Q3","Q4"]]
stock_index_excess = stock_index[["Q1", "Q2", "Q3", "Q4"]] - money_market[["Q1", "Q2", "Q3","Q4"]]
int_value_excess = int_value[["Q1", "Q2", "Q3", "Q4"]] - money_market[["Q1", "Q2", "Q3","Q4"]]
reit_excess = reit[["Q1", "Q2", "Q3", "Q4"]] - money_market[["Q1", "Q2", "Q3","Q4"]]

In [8]:
# a demo
print(bond_index_excess)

        Q1      Q2      Q3      Q4
0  -0.0186 -0.0065 -0.0051  0.0102
1   0.0070  0.0122  0.0044  0.0008
2   0.0298  0.0223  0.0027 -0.0334
3   0.0163 -0.0181  0.0115 -0.0065
4   0.0190  0.0196  0.0016  0.0171
5  -0.0008 -0.0243  0.0052 -0.0020
6   0.0024  0.0213  0.0152  0.0010
7   0.0022  0.0223  0.0398  0.0094
8   0.0168  0.0357  0.0239 -0.0138
9  -0.0001  0.0163  0.0361  0.0003
10  0.0121 -0.0169 -0.0098  0.0372
11  0.0017 -0.0194  0.0162  0.0185


In [11]:
# reshape to make a vector
bond_excess_vec = np.array(bond_index_excess[["Q1", "Q2", "Q3", "Q4"]]).reshape((-1,1))
np.cov(bond_excess_vec.T, bias = True)

array(0.00027117)

In [12]:
# the professor forgets to divide the S.T @ S by the number of observations. If you multiply the above result by 48, you get 

In [13]:
0.00027117*48

0.013016159999999999